In [47]:
import praw
import pandas as pd
from praw.models import MoreComments

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps


# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

# df1 = pd.DataFrame({
#     'post_id': ['p1', 'p2', 'p3'],
#     'title': ['Title A', 'Title B', 'Title C'],
#     'author': ['user1', 'user2', 'user3']
# })

# df2 = pd.DataFrame({
#     'comment_id': ['c1', 'c2', 'c3', 'c4', 'c5'],
#     'post_id': ['p1', 'p1', 'p2', 'p2', 'p4'],  # Note: p4 doesn't exist in df1
#     'comment': ['Great post!', 'Thanks!', 'Interesting', 'I agree', 'Nice!']
# })
# outer_merged = df1.merge(df2, on='post_id', how='outer')
# print(outer_merged)

In [48]:
consulting_posts = subreddit.search('consulting')

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [49]:
for post in consulting_posts:
    #The title of the post
    posts_dict['Title'].append(post.title)
    #The text inside of the post
    posts_dict['Post Text'].append(post.selftext)
    #Unique ID of each post
    posts_dict['ID'].append(post.id)
    #The scoure of a post
    posts_dict['Score'].append(post.score)
    #Total number of comments inside the post
    posts_dict['Total Comments'].append(post.num_comments)
    #URL of each post
    posts_dict['Post URL'].append(post.url)

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,266,112,https://www.reddit.com/r/berkeley/comments/16o...
1,What are consulting clubs actually looking for?,I am an incoming freshman at Haas next year an...,1kvi9jt,42,55,https://www.reddit.com/r/berkeley/comments/1kv...
2,Tech PM blocks all “.berkeley.edu” e-mails bc ...,Consulting clubs making us look bad smh,19d8ceg,561,43,https://i.redd.it/em6d28vzh2ec1.jpeg
3,Rejected from consulting club after having bee...,titles kinda self explanatory lol. didn’t even...,1na99k8,72,22,https://www.reddit.com/r/berkeley/comments/1na...
4,Why are some of these Business/Consulting Club...,My friend and I were enjoying dinner in the so...,16jsy19,367,55,https://www.reddit.com/r/berkeley/comments/16j...
...,...,...,...,...,...,...
95,another bear rejected by consulting clubs,I know it's just a club but man I wanna cry i ...,piudil,28,7,https://www.reddit.com/r/berkeley/comments/piu...
96,There are 150-odd chapters of Chinese Students...,,69fs7e,50,22,https://www.nytimes.com/2017/05/04/us/chinese-...
97,NEW! - Stress Management Consultations,Stress Management Consultations at the UHS Car...,zdf1h4,5,0,https://www.reddit.com/r/berkeley/comments/zdf...
98,Made it to a Consulting Club’s Final Round Int...,I’m not happy about getting rejected but I did...,sjnb6y,3,6,https://www.reddit.com/r/berkeley/comments/sjn...


In [50]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Diversatech at Berkeley is the worst one,133
1,Bro there is asian monoculture at the school lol,180
2,"Interesting take and valid points, but i think...",149
3,for the pbl one they have basically a DEI stat...,64
4,sucks but it’s how it be sometimes - you can m...,67
...,...,...
577,"Hey there, just went through the other side of...",2
578,Hi! I went through all the interview rounds bu...,1
579,Start your own if you really want to be one of...,1
580,Tailor it to the club as much as possible. We ...,1


In [51]:
import ollama
# embed_model = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'    Can't take large enough data
embed_model = 'hf.co/bartowski/granite-embedding-30m-english-GGUF'
lang_model = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

ollama.pull(embed_model)
ollama.pull(lang_model)

consulting_club_posts['embedding'] = consulting_club_posts['Post Text'].apply(
    lambda t: ollama.embed(model=embed_model, input=t)['embeddings'])

comments_df['embedding'] = comments_df['Comment'].apply(
    lambda t: ollama.embed(model=embed_model, input=t)['embeddings'])

# Formats it so each element is not a list of a list unnecessarily
consulting_club_posts['embedding'] = consulting_club_posts['embedding'].apply(lambda x: x[0] if len(x) == 1 else x)
comments_df['embedding'] = comments_df['embedding'].apply(lambda x: x[0] if len(x) == 1 else x)


In [52]:
# Creating weights for each embedding based on score values, plugging them into a tuned sigmoid function
# 0-200 has factor ~1, 200-500 has factor ~1.3, 500-700 has factor ~1.6, 700+ has factor ~1.8
# Doesn't function well, need to include comment score scaling as well if implemented

# consulting_club_posts['Score Factor'] = consulting_club_posts['Score'].apply(lambda x: 1 + 1/(1+20*pow(2, -x/100)))

# consulting_club_posts['Weighted embedding'] = consulting_club_posts['embedding'].combine(consulting_club_posts['Score Factor'], lambda lst, weight: [weight * x for x in lst])
# print(consulting_club_posts['Weighted embedding'], consulting_club_posts['Score'])

In [53]:
def cosine_similarity(a, b):
    dot_prod = sum([x * y for x, y in zip(a, b)])
    mag_a = pow(sum([pow(x, 2) for x in a]), 0.5)
    mag_b = pow(sum([pow(y, 2) for y in b]), 0.5)
    return mag_a * mag_b and (dot_prod) / (mag_a * mag_b)   # add mag_b to dot_prod for weightage

In [54]:
def retrieve_data(query, n=5):
    query_embed = ollama.embed(model=embed_model, input=query)['embeddings']
    similarities = consulting_club_posts['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))     # Change to 'weighted embedding' for weightage
    pd.concat([similarities, comments_df['embedding'].apply(lambda x: cosine_similarity(query_embed[0], x))])   
    return similarities.nlargest(n)

In [55]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve_data(input_query)


instruction_prompt = f'''You are a helpful chatbot aimed to help UC Berkeley students learn about and choose clubs to join.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {consulting_club_posts.loc[i, 'Post Text']}' for i in retrieved_knowledge.index])}
'''

print(instruction_prompt)

You are a helpful chatbot aimed to help UC Berkeley students learn about and choose clubs to join.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - I'm a freshman EECS major at Cal, and didn't end up joining any clubs my first semester because I wanted to focus on generally getting used to Cal and getting an internship for the summer. Now, however, I kinda realize that my social circle/network isn't very big and I honestly want to meet more people involved with tech, and have a better overall social scene.

Fortunately, I was able to get a FAANG internship for the summer, so I'm not too worried about resume value etc, and I'm more interested in working on **genuinely** cool projects and having a good time.

I know a lot of the tech/consulting clubs here have a reputation for being prestige competitions, but I'm wondering if there are any that actually do interesting work(that don't require too much knowledge beforehand). Currently I

In [56]:
stream = ollama.chat(
  model=lang_model,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
Here's some general information about the tech-specific consulting clubs at UC Berkeley:

* **Berkeley Consulting**: A well-established club that provides consulting services to clients. While it's considered prestigious, getting into it is competitive, and you'll likely need a strong understanding of business and operations.
* **Voyager Consulting**: The "vanguard" of consulting clubs on campus, known for their innovative approaches and high demand among top companies.
* **Venture Strategy Solutions (VSS)**: A highly respected club that focuses on venture capital analysis, strategy, and team management. They're often considered one of the most prestigious consulting clubs at UC Berkeley.

To increase your chances of getting into these clubs, consider the following:

1. **Genuinely impressive projects**: Many successful consultants start with smaller projects or initiatives that demonstrate their capabilities and values.
2. **Strong internship experience**: If you're 